<a href="https://colab.research.google.com/github/SALMANKHANPM/AI-Powered-Student-Assistance-Chatbot/blob/main/nb/Gemma3_(4B).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

Unsloth now supports Text-to-Speech (TTS) models. Read our [guide here](https://docs.unsloth.ai/basics/text-to-speech-tts-fine-tuning).

Read our **[Qwen3 Guide](https://docs.unsloth.ai/basics/qwen3-how-to-run-and-fine-tune)** and check out our new **[Dynamic 2.0](https://docs.unsloth.ai/basics/unsloth-dynamic-2.0-ggufs)** quants which outperforms other quantization methods!

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

### Unsloth

`FastModel` supports loading nearly any model now! This includes Vision and Text models!

In [2]:
from unsloth import FastModel
import torch

fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",

    # Other popular models!
    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/Llama-3.3-70B",
    "unsloth/mistral-7b-instruct-v0.3",
    "unsloth/Phi-4",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it",
    max_seq_length = 2048, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.6.2: Fast Gemma3 patching. Transformers: 4.52.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


model.safetensors:   0%|          | 0.00/4.56G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update a small amount of parameters!

In [3]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 8,           # Larger = higher accuracy, but might overfit
    lora_alpha = 8,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.model.language_model` require gradients


<a name="Data"></a>
### Data Prep
We now use the `Gemma-3` format for conversation style finetunes. We use [Maxime Labonne's FineTome-100k](https://huggingface.co/datasets/mlabonne/FineTome-100k) dataset in ShareGPT style. Gemma-3 renders multi turn conversations like below:

```
<bos><start_of_turn>user
Hello!<end_of_turn>
<start_of_turn>model
Hey there!<end_of_turn>
```

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, llama3, phi4, qwen2.5, gemma3` and more.

In [4]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

In [7]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
dataset = load_dataset("maya-research/IndicVault", "Telugu")

In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'response'],
        num_rows: 93028
    })
})

In [11]:
def convert_to_sharegpt(dataset):
    """Convert question-response dataset to ShareGPT format"""

    def transform_row(example):
        return {
            "conversations": [
                {
                    "from": "human",
                    "value": example["question"]
                },
                {
                    "from": "gpt",
                    "value": example["response"]
                }
            ]
        }

    return dataset.map(transform_row, remove_columns=["question", "response"])

# Apply the transformation
sharegpt_dataset = convert_to_sharegpt(dataset)

# The result will have this structure:
# Dataset({
#     features: ['conversations'],
#     num_rows: 93028
# })

Map:   0%|          | 0/93028 [00:00<?, ? examples/s]

In [17]:
sharegpt_dataset['train'][0]

{'conversations': [{'from': 'human',
   'value': 'ఇండియాలో గ్రోసరీస్ మీద డబ్బులు సేవ్ చేయడానికి బెస్ట్ వేస్ ఏంటి?\n'},
  {'from': 'gpt',
   'value': 'ఇండియాలో గ్రోసరీస్ మీద డబ్బులు సేవ్ చేయడానికి చాలా బెస్ట్ వేస్ ఉన్నాయి. ఇక్కడ కొన్ని టిప్స్ ఉన్నాయి చూడండి:\n\n1. **ముందుగానే ప్లాన్ చేసుకోండి:** షాపింగ్ కి వెళ్ళేముందు ఏం కావాలో లిస్ట్ ప్రిపేర్ చేసుకోండి. లిస్ట్ ఉంటే, అనవసరమైనవి కొనకుండా ఉంటారు, డబ్బులు సేవ్ అవుతాయి.\n\n2. **ప్రైస్ కంపేర్ చేయండి:**  ఒకే ప్రోడక్ట్ వేర్వేరు షాపుల్లో వేర్వేరు రేట్లు ఉండొచ్చు. సూపర్\u200cమార్కెట్స్, లోకల్ షాప్స్, ఆన్\u200cలైన్ స్టోర్స్ లో ప్రైస్ కంపేర్ చేసి ఎక్కడ తక్కువ ఉంటే అక్కడ కొనండి.\n\n3. **సేల్స్ అండ్ డిస్కౌంట్స్ చూడండి:**  సూపర్\u200cమార్కెట్స్ లో వీక్లీ సేల్స్, మంత్లీ ఆఫర్స్ ఉంటాయి. వాటిని ఫాలో అవ్వండి. ఫెస్టివల్ సేల్స్ లో కూడా మంచి డిస్కౌంట్స్ దొరుకుతాయి.\n\n4. **బల్క్ లో కొనండి:**  రైస్, పప్పులు, నూనె లాంటివి ఎక్కువ క్వాంటిటీలో కొనడం వల్ల రేట్ తక్కువ పడుతుంది. ఫ్యామిలీ పెద్దది అయితే ఇది చాలా యూస్\u200cఫుల్ టిప్.\n\n5. **సీజనల్ ఫ్రూట్స్ అండ్ వెజిటే

We now use `standardize_data_formats` to try converting datasets to the correct format for finetuning purposes!

In [18]:
from unsloth.chat_templates import standardize_data_formats
dataset = standardize_data_formats(sharegpt_dataset['train'])

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/93028 [00:00<?, ? examples/s]

Let's see how row 100 looks like!

In [21]:
dataset[10]

{'conversations': [{'content': 'ఇండియాలో హెల్త్ కేర్ కాస్ట్స్ తగ్గించడానికి ఏమైనా స్ట్రాటజీస్ ఉన్నాయా?\n',
   'role': 'user'},
  {'content': 'భారతదేశంలో హెల్త్ కేర్ ఖర్చులు తగ్గించడానికి కొన్ని స్ట్రాటజీస్ ఉన్నాయి, చూడు:\n\n1.  **ప్రివెన్షన్ మేలు:**  జబ్బులు రాకుండా చూసుకోవడం చాలా ఇంపార్టెంట్. హెల్త్ ఎడ్యుకేషన్ పెంచితే, ప్రజలు హెల్తీ లైఫ్ స్టైల్ ఫాలో అవుతారు. యోగా, ఎక్సర్సైజ్ లాంటివి రెగ్యులర్ గా చేస్తే చాలా రోగాలు రాకుండా కాపాడుకోవచ్చు. ఇది హాస్పిటల్ ఖర్చుల్ని బాగా తగ్గిస్తుంది.\n\n2.  **గవర్నమెంట్ స్కీమ్స్:**  ప్రభుత్వం చాలా హెల్త్ స్కీమ్స్ పెట్టింది. ఆయుష్మాన్ భారత్ లాంటివి పేద ప్రజలకి ఫ్రీ ట్రీట్మెంట్ ఇస్తున్నాయి. ఇలాంటి స్కీమ్స్ ని ఇంకా బాగా చేస్తే, అందరికీ తక్కువ ఖర్చుతో ట్రీట్మెంట్ అందుతుంది.\n\n3.  **టెక్నాలజీ వాడకం:**  టెక్నాలజీతో హెల్త్ కేర్ ని చీప్ చేయొచ్చు. టెలిమెడిసిన్ ద్వారా డాక్టర్లు దూరం నుండి పేషెంట్స్\u200cకి ట్రీట్మెంట్ ఇవ్వొచ్చు. దీనివల్ల ట్రావెల్ ఖర్చులు తగ్గుతాయి, టైం సేవ్ అవుతుంది. AI టెక్నాలజీ వాడితే, రోగాల్ని త్వరగా కనిపెట్టొచ్చు, ట్రీట్మెంట్ ఫాస్ట్ గా స్టార్ట్

We now have to apply the chat template for `Gemma-3` onto the conversations, and save it to `text`. We remove the `<bos>` token using removeprefix(`'<bos>'`) since we're finetuning. The Processor will add this token before training and the model expects only one.

In [22]:
def formatting_prompts_func(examples):
   convos = examples["conversations"]
   texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False).removeprefix('<bos>') for convo in convos]
   return { "text" : texts, }

dataset = dataset.map(formatting_prompts_func, batched = True)

Map:   0%|          | 0/93028 [00:00<?, ? examples/s]

Let's see how the chat template did! Notice there is no `<bos>` token as the processor tokenizer will be adding one.

In [26]:
dataset[10]["text"]

'<start_of_turn>user\nఇండియాలో హెల్త్ కేర్ కాస్ట్స్ తగ్గించడానికి ఏమైనా స్ట్రాటజీస్ ఉన్నాయా?<end_of_turn>\n<start_of_turn>model\nభారతదేశంలో హెల్త్ కేర్ ఖర్చులు తగ్గించడానికి కొన్ని స్ట్రాటజీస్ ఉన్నాయి, చూడు:\n\n1.  **ప్రివెన్షన్ మేలు:**  జబ్బులు రాకుండా చూసుకోవడం చాలా ఇంపార్టెంట్. హెల్త్ ఎడ్యుకేషన్ పెంచితే, ప్రజలు హెల్తీ లైఫ్ స్టైల్ ఫాలో అవుతారు. యోగా, ఎక్సర్సైజ్ లాంటివి రెగ్యులర్ గా చేస్తే చాలా రోగాలు రాకుండా కాపాడుకోవచ్చు. ఇది హాస్పిటల్ ఖర్చుల్ని బాగా తగ్గిస్తుంది.\n\n2.  **గవర్నమెంట్ స్కీమ్స్:**  ప్రభుత్వం చాలా హెల్త్ స్కీమ్స్ పెట్టింది. ఆయుష్మాన్ భారత్ లాంటివి పేద ప్రజలకి ఫ్రీ ట్రీట్మెంట్ ఇస్తున్నాయి. ఇలాంటి స్కీమ్స్ ని ఇంకా బాగా చేస్తే, అందరికీ తక్కువ ఖర్చుతో ట్రీట్మెంట్ అందుతుంది.\n\n3.  **టెక్నాలజీ వాడకం:**  టెక్నాలజీతో హెల్త్ కేర్ ని చీప్ చేయొచ్చు. టెలిమెడిసిన్ ద్వారా డాక్టర్లు దూరం నుండి పేషెంట్స్\u200cకి ట్రీట్మెంట్ ఇవ్వొచ్చు. దీనివల్ల ట్రావెల్ ఖర్చులు తగ్గుతాయి, టైం సేవ్ అవుతుంది. AI టెక్నాలజీ వాడితే, రోగాల్ని త్వరగా కనిపెట్టొచ్చు, ట్రీట్మెంట్ ఫాస్ట్ గా స్టార్ట్ చేయొచ్చు.\n\

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.

In [32]:

!pip install wandb
import wandb
#wb_token = user_secrets.get_secret('wandb')

wandb.login(key='84aa8e56cbd4d135185eb018719feed8eac9e7c0')

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: salmankhanpm (salmankhanpm-kl-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [33]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 30,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "wandb", # Use this for WandB etc
        dataset_num_proc=2,
    ),
)

Unsloth: Switching to float32 training since model cannot work with float16


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/93028 [00:00<?, ? examples/s]

We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs. This helps increase accuracy of finetunes!

In [34]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

Map (num_proc=2):   0%|          | 0/93028 [00:00<?, ? examples/s]

Let's verify masking the instruction part is done! Let's print the 100th row again.  Notice how the sample only has a single `<bos>` as expected!

In [35]:
tokenizer.decode(trainer.train_dataset[101]["input_ids"])

'<bos><start_of_turn>user\nక్రెడిట్ కార్డ్స్ మీద వచ్చే రివార్డ్ ప్రోగ్రామ్స్ ని ఎలా మాక్సిమైజ్ చేసుకోవచ్చు?<end_of_turn>\n<start_of_turn>model\nERROR: No API keys available after retries<end_of_turn>\n'

Now let's print the masked out example - you should see only the answer is present:

In [36]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[101]["labels"]]).replace(tokenizer.pad_token, " ")

'                                    ERROR: No API keys available after retries<end_of_turn>\n'

In [37]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
5.57 GB of memory reserved.


Let's train the model! To resume a training run, set `trainer.train(resume_from_checkpoint = True)`

In [39]:
# Move model to GPU and set dtype (usually handled by SFTTrainer, but double checking)
model.to("cuda") # SFTTrainer should handle this
for param in model.parameters():
     if param.dtype == torch.float32:
         param.data = param.data.to(torch.float16) # Potentially problematic with 4bit

# Train the model
#trainer_stats = trainer.train()

In [40]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 93,028 | Num Epochs = 1 | Total steps = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 14,901,248/4,000,000,000 (0.37% trained)


RuntimeError: expected mat1 and mat2 to have the same dtype, but got: float != c10::Half

In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

<a name="Inference"></a>
### Inference
Let's run the model via Unsloth native inference! According to the `Gemma-3` team, the recommended settings for inference are `temperature = 1.0, top_p = 0.95, top_k = 64`

In [ ]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)
messages = [{
    "role": "user",
    "content": [{
        "type" : "text",
        "text" : "Continue the sequence: 1, 1, 2, 3, 5, 8,",
    }]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)
outputs = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
)
tokenizer.batch_decode(outputs)

['<bos><start_of_turn>user\nContinue the sequence: 1, 1, 2, 3, 5, 8,<end_of_turn>\n<start_of_turn>model\n13, 21, 34, 55, 89...\n\nThis is the Fibonacci sequence, where each number is the sum of the two preceding ones.\n<end_of_turn>']

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "Why is the sky blue?",}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

Okay, let's break down why the sky is blue! It's a fascinating phenomenon that boils down to a combination of physics and light. Here's the explanation:

**1. Sunlight and its Colors:**

* Sunlight, which appears white to us, is actually made up of *all* the


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("gemma-3")  # Local saving
tokenizer.save_pretrained("gemma-3")
# model.push_to_hub("HF_ACCOUNT/gemma-3", token = "...") # Online saving
# tokenizer.push_to_hub("HF_ACCOUNT/gemma-3", token = "...") # Online saving

['gemma-3/processor_config.json']

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastModel
    model, tokenizer = FastModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 2048,
        load_in_4bit = True,
    )

messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "What is Gemma-3?",}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

Okay, let's break down what Gemma-3 is. It's a fascinating development in the world of AI, and here's a comprehensive overview:

**1. What it is:**

* **A Family of Open-Weight Language Models:** Gemma-3 isn't just *one* model


### Saving to float16 for VLLM

We also support saving to `float16` directly for deployment! We save it in the folder `gemma-3-finetune`. Set `if False` to `if True` to let it run!

In [ ]:
if False: # Change to True to save finetune!
    model.save_pretrained_merged("gemma-3-finetune", tokenizer)

If you want to upload / push to your Hugging Face account, set `if False` to `if True` and add your Hugging Face token and upload location!

In [ ]:
if False: # Change to True to upload finetune
    model.push_to_hub_merged(
        "HF_ACCOUNT/gemma-3-finetune", tokenizer,
        token = "hf_..."
    )

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now for all models! For now, you can convert easily to `Q8_0, F16 or BF16` precision. `Q4_K_M` for 4bit will come later!

In [ ]:
if False: # Change to True to save to GGUF
    model.save_pretrained_gguf(
        "gemma-3-finetune",
        quantization_type = "Q8_0", # For now only Q8_0, BF16, F16 supported
    )

Likewise, if you want to instead push to GGUF to your Hugging Face account, set `if False` to `if True` and add your Hugging Face token and upload location!

In [ ]:
if False: # Change to True to upload GGUF
    model.push_to_hub_gguf(
        "gemma-3-finetune",
        quantization_type = "Q8_0", # Only Q8_0, BF16, F16 supported
        repo_id = "HF_ACCOUNT/gemma-finetune-gguf",
        token = "hf_...",
    )

Now, use the `gemma-3-finetune.gguf` file or `gemma-3-finetune-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
